In [1]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [2]:
data = pd.read_csv('creditcard.csv')

print(data.head(10))

   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   
5   2.0 -0.425966  0.960523  1.141109 -0.168252  0.420987 -0.029728  0.476201   
6   4.0  1.229658  0.141004  0.045371  1.202613  0.191881  0.272708 -0.005159   
7   7.0 -0.644269  1.417964  1.074380 -0.492199  0.948934  0.428118  1.120631   
8   7.0 -0.894286  0.286157 -0.113192 -0.271526  2.669599  3.721818  0.370145   
9   9.0 -0.338262  1.119593  1.044367 -0.222187  0.499361 -0.246761  0.651583   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -

In [14]:
X = data.drop(['Class'], axis=1)
y = data['Class']

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(284807, 30)
(284807,)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [17]:
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
X_test_torch  = torch.tensor(X_test,  dtype=torch.float32)
y_test_torch  = torch.tensor(y_test,  dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset  = TensorDataset(X_test_torch,  y_test_torch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)

In [28]:
n_qubits = 8
n_layers = 1

dev = qml.device('lightning.gpu', wires=n_qubits)

@qml.qnode(dev, interface="torch")
def qlayer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=list(np.arange(n_qubits)), rotation='X')
    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

In [29]:
class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 128)
        self.fc2 = nn.Linear(128, n_qubits)
        self.scale = nn.Parameter(torch.tensor([2 * np.pi]))
        self.qnn_weights = nn.Parameter(torch.rand(n_layers, n_qubits, 3) * 1e-3)
        self.output_layer = nn.Linear(n_qubits, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(x) * self.scale
        batch_size = x.size(0)
        out = torch.empty((batch_size, n_qubits), dtype=torch.float32, device=x.device)
        for batch_idx in range(batch_size):
            expvals = qlayer(x[batch_idx], self.qnn_weights)
            out[batch_idx] = torch.stack(expvals)
        x = self.output_layer(out)
        return x

In [19]:
class ClassicalClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(len(X.T), 128)
        self.fc2 = nn.Linear(128, 8)
        self.output_layer = nn.Linear(8, 2)  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output_layer(x)
        return x

In [21]:
sample_size_for_each_class_classical = 5
batch_size_classical = 32

X_train_sampled_classical = []
y_train_sampled_classical = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=sample_size_for_each_class_classical, replace=False)

    X_train_sampled_classical.extend(X_train[sampled_indices])
    y_train_sampled_classical.extend(y_train[sampled_indices]) 

X_train_sampled_classical = np.array(X_train_sampled_classical)
y_train_sampled_classical = np.array(y_train_sampled_classical)

X_train_sampled_torch_classical = torch.tensor(X_train_sampled_classical, dtype=torch.float32)
y_train_sampled_torch_classical = torch.tensor(y_train_sampled_classical, dtype=torch.long)

train_sampled_dataset_classical = TensorDataset(X_train_sampled_torch_classical, y_train_sampled_torch_classical)
train_sampled_loader_classical = DataLoader(train_sampled_dataset_classical, batch_size=batch_size_classical, shuffle=True)

In [30]:
sample_size_for_each_class_quantum = 5
batch_size_quantum = 32

X_train_sampled_quantum = []
y_train_sampled_quantum = []


for class_label in np.unique(y_train):
    class_indices = np.where(y_train == class_label)[0]
    sampled_indices = np.random.choice(class_indices, size=sample_size_for_each_class_quantum, replace=False)

    X_train_sampled_quantum.extend(X_train[sampled_indices])
    y_train_sampled_quantum.extend(y_train[sampled_indices]) 

X_train_sampled_quantum = np.array(X_train_sampled_quantum)
y_train_sampled_quantum = np.array(y_train_sampled_quantum)

X_train_sampled_torch_quantum = torch.tensor(X_train_sampled_quantum, dtype=torch.float32)
y_train_sampled_torch_quantum = torch.tensor(y_train_sampled_quantum, dtype=torch.long)

train_sampled_dataset_quantum = TensorDataset(X_train_sampled_torch_quantum, y_train_sampled_torch_quantum)
train_sampled_loader_quantum = DataLoader(train_sampled_dataset_quantum, batch_size=batch_size_quantum, shuffle=True)

In [31]:
model_quantum = HybridClassifier()

learning_rate_quantum = 1e-3
batch_size_quantum = 32
epochs_quantum = 3

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop_quantum(dataloader, loss_fn, optimizer):
    model_quantum.train()
    model_quantum.to(torch_device)  
    for _, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model_quantum(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test_loop_quantum(dataloader, loss_fn):
    model_quantum.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model_quantum.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model_quantum(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size_quantum) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [22]:
model_classical = ClassicalClassifier()

learning_rate_classical = 1e-3
batch_size_classical = 32
epochs_classical = 20

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

def train_loop_classical(dataloader, loss_fn, optimizer):
    model_classical.train()
    model_classical.to(torch_device)  
    for _, (X, y) in enumerate(dataloader):
        X, y = X.to(torch_device), y.to(torch_device)
        pred = model_classical(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


def test_loop_classical(dataloader, loss_fn):
    model_classical.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    model_classical.to(torch_device)  
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(torch_device), y.to(torch_device)
            pred = model_classical(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()

    print(f"Accuracy: {correct / (len(dataloader) * batch_size_classical) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")

    return test_loss

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_classical.parameters(), lr=learning_rate_classical)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data for classical\n--------------------------")
for t in range(epochs_classical):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_classical(train_sampled_loader_classical, loss_fn, optimizer)
    test_loss = test_loop_classical(test_loader, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_classical(test_loader, loss_fn)

Before training sampled data for classical
--------------------------
Epoch 1
--------------------------
Accuracy: 67.91128540039062
Avg loss: 0.667476 

Epoch 2
--------------------------
Accuracy: 71.08892822265625
Avg loss: 0.661653 

Epoch 3
--------------------------
Accuracy: 74.12093353271484
Avg loss: 0.655290 

Epoch 4
--------------------------
Accuracy: 77.27049255371094
Avg loss: 0.648201 

Epoch 5
--------------------------
Accuracy: 79.7024154663086
Avg loss: 0.641866 

Epoch 6
--------------------------
Accuracy: 82.02555084228516
Avg loss: 0.635391 

Epoch 7
--------------------------
Accuracy: 84.41535949707031
Avg loss: 0.628393 

Epoch 8
--------------------------
Accuracy: 86.84552001953125
Avg loss: 0.620529 

Epoch 9
--------------------------
Accuracy: 88.80720520019531
Avg loss: 0.613202 

Epoch 10
--------------------------
Accuracy: 90.38812255859375
Avg loss: 0.606320 

Epoch 11
--------------------------
Accuracy: 91.73042297363281
Avg loss: 0.599642 

Epoch

0.5289009970470067

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_classical.parameters(), lr=learning_rate_classical)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training for classical\n--------------------------")
for t in range(epochs_classical):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_classical(train_loader, loss_fn, optimizer)
    test_loss = test_loop_classical(test_loader, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_classical(test_loader, loss_fn)
print("Done!")

Before training for classical
--------------------------
Epoch 1
--------------------------
Accuracy: 99.88243865966797
Avg loss: 0.002916 

Epoch 2
--------------------------
Accuracy: 99.87718200683594
Avg loss: 0.003066 

Epoch 3
--------------------------
Accuracy: 99.87718200683594
Avg loss: 0.003122 

Epoch 4
--------------------------
Accuracy: 99.88069152832031
Avg loss: 0.002804 

Epoch 5
--------------------------
Accuracy: 99.87718200683594
Avg loss: 0.002743 

Epoch 6
--------------------------
Accuracy: 99.87542724609375
Avg loss: 0.003090 

Epoch 7
--------------------------
Accuracy: 99.88420104980469
Avg loss: 0.002847 

Epoch 8
--------------------------
Accuracy: 99.86314392089844
Avg loss: 0.003249 

Epoch 9
--------------------------
Accuracy: 99.88243865966797
Avg loss: 0.002877 

Epoch 10
--------------------------
Accuracy: 99.88069152832031
Avg loss: 0.002911 

Epoch 11
--------------------------
Accuracy: 99.87016296386719
Avg loss: 0.003120 

Epoch 12
--------

In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_quantum.parameters(), lr=learning_rate_quantum)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)
start_time = time.time()

print(f"Before training sampled data for quantum\n--------------------------")
for t in range(epochs_quantum):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_quantum(train_sampled_loader_quantum, loss_fn, optimizer)
    test_loss = test_loop_quantum(test_loader, loss_fn)
    scheduler.step(test_loss)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_quantum(test_loader, loss_fn)

Before training sampled data for quantum
--------------------------
Epoch 1
--------------------------
Accuracy: 49.04548263549805
Avg loss: 0.723206 

Epoch 2
--------------------------
Accuracy: 54.41816329956055
Avg loss: 0.695978 

Epoch 3
--------------------------
Accuracy: 58.6661262512207
Avg loss: 0.678617 

Training took 36.64 minutes.
Accuracy: 58.6661262512207
Avg loss: 0.678617 



0.6786167317063269

In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_quantum.parameters(), lr=learning_rate_quantum)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training for quantum\n--------------------------")
for t in range(epochs_quantum):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop_quantum(train_loader, loss_fn, optimizer)
    test_loss = test_loop_quantum(test_loader, loss_fn)
    scheduler.step(test_loss)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")
test_loop_quantum(test_loader, loss_fn)

Before training for quantum
--------------------------
Epoch 1
--------------------------


KeyboardInterrupt: 